In [ ]:
## Do
# plot contribution to j0rad integral --> rewrite j0_rad function
# modify V_loss_Rau outputs, so it gives all voltages (Eref,Voc_SQ,Voc_jsc,Voc_rad,Voc) and all Deltas (Delta_SQ,Delta_jsc,Deta_rad,Delta_nonrad)

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit
from scipy import integrate as ig
from functions import J0, J0_rad, SQ, Voc_rad, LED_QE, calculate_summary, Vloss_CT, Vloss_SQ
from utils import bb, getAM15, linear, set_up_plot

In [ ]:
T = 293 # [K] ambient & cell temperature
h = 6.626 * 10**(-34) # [kgm^2/s]
h_eV = 4.1357*10**(-15) # eV s
c = 2.998 * 10**(8) # [m/s]
k = 1.3806 * 10**(-23) # [kgm^2/s^2K]
k_eV = 8.6173*10**(-5) # eV / K
q = 1.60217662 * 10**(-19) # [C]
q_eV = 1
Vth = k_eV*T/q_eV # thermal voltage [eV]

In [ ]:
codePath = os.getcwd()#only makes sense after kernel restart
codePath

In [ ]:
os.chdir('C:/Users/pasca/OneDrive - Nexus365/Post/data/')#/home/jungbluth/Desktop/
os.getcwd()

### Backup from functions

In [ ]:
#def getAM15():
    #filePath = codePath + '\ASTMG173.csv'# 1:wavelength [nm], 3:AM15g spectral power [W/(m^2nm)]
    

### Calculate Relevant Parameters

In [ ]:
#summary